## Introduktion til Regression - Lektion 1

#### Sæt det i perspektiv

✅ Der findes mange typer regressionsmetoder, og hvilken du vælger afhænger af det svar, du søger. Hvis du vil forudsige den sandsynlige højde for en person med en given alder, vil du bruge `lineær regression`, da du søger en **numerisk værdi**. Hvis du er interesseret i at finde ud af, om en type køkken skal betragtes som vegansk eller ej, leder du efter en **kategoriinddeling**, så du vil bruge `logistisk regression`. Du vil lære mere om logistisk regression senere. Tænk lidt over nogle spørgsmål, du kan stille til data, og hvilken af disse metoder der ville være mest passende.

I denne sektion vil du arbejde med et [lille datasæt om diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Forestil dig, at du ville teste en behandling for diabetiske patienter. Maskinlæringsmodeller kan hjælpe dig med at afgøre, hvilke patienter der vil reagere bedre på behandlingen, baseret på kombinationer af variabler. Selv en meget grundlæggende regressionsmodel, når den visualiseres, kan vise information om variabler, der kan hjælpe dig med at organisere dine teoretiske kliniske forsøg.

Med det sagt, lad os komme i gang med denne opgave!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>

<!--![Kunstværk af \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Kunstværk af @allison_horst-->


## 1. Indlæsning af vores værktøjssæt

Til denne opgave skal vi bruge følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling af R-pakker](https://www.tidyverse.org/packages), der er designet til at gøre datavidenskab hurtigere, nemmere og sjovere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er en [samling af pakker](https://www.tidymodels.org/packages/) til modellering og maskinlæring.

Du kan installere dem med følgende kommando:

`install.packages(c("tidyverse", "tidymodels"))`

Scriptet nedenfor tjekker, om du har de nødvendige pakker til at fuldføre dette modul, og installerer dem for dig, hvis nogle mangler.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Lad os nu indlæse disse fantastiske pakker og gøre dem tilgængelige i vores nuværende R-session. (Dette er kun for illustration, `pacman::p_load()` har allerede gjort det for dig)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabetes-datasættet

I denne øvelse vil vi demonstrere vores regressionsevner ved at lave forudsigelser på et diabetes-datasæt. [Diabetes-datasættet](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) indeholder `442 prøver` med data om diabetes, med 10 prædiktorvariabler: `alder`, `køn`, `body mass index`, `gennemsnitligt blodtryk` og `seks blodserummålinger` samt en resultatvariabel `y`: et kvantitativt mål for sygdomsprogression ét år efter baseline.

|Antal observationer|442|
|-------------------|:---|
|Antal prædiktorer|De første 10 kolonner er numeriske prædiktive|
|Resultat/Mål|Kolonne 11 er et kvantitativt mål for sygdomsprogression ét år efter baseline|
|Information om prædiktorer|- alder i år
||- køn
||- bmi body mass index
||- bp gennemsnitligt blodtryk
||- s1 tc, total serumkolesterol
||- s2 ldl, lavdensitetslipoproteiner
||- s3 hdl, højdensitetslipoproteiner
||- s4 tch, total kolesterol / HDL
||- s5 ltg, muligvis logaritmen af serum triglyceridniveau
||- s6 glu, blodsukkerniveau|

> 🎓 Husk, dette er superviseret læring, og vi har brug for et navngivet 'y'-mål.

Før du kan manipulere data med R, skal du importere dataene til R's hukommelse eller oprette en forbindelse til dataene, som R kan bruge til at få adgang til dem eksternt.

> [readr](https://readr.tidyverse.org/)-pakken, som er en del af Tidyverse, giver en hurtig og brugervenlig måde at læse rektangulære data ind i R.

Lad os nu indlæse diabetes-datasættet fra denne kilde-URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Vi vil også udføre en kontrol af vores data ved hjælp af `glimpse()` og vise de første 5 rækker med `slice()`.

Før vi går videre, lad os introducere noget, du ofte vil støde på i R-kode 🥁🥁: pipe-operatoren `%>%`

Pipe-operatoren (`%>%`) udfører operationer i logisk rækkefølge ved at sende et objekt videre til en funktion eller et kald. Du kan tænke på pipe-operatoren som at sige "og derefter" i din kode.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` viser os, at denne data har 442 rækker og 11 kolonner, hvor alle kolonnerne er af datatypen `double`.

<br>

> glimpse() og slice() er funktioner i [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, en del af Tidyverse, er en grammatik for datamanipulation, der tilbyder et konsistent sæt af verber, som hjælper dig med at løse de mest almindelige udfordringer inden for datamanipulation.

<br>

Nu hvor vi har dataen, lad os fokusere på én feature (`bmi`) som mål for denne øvelse. Dette kræver, at vi vælger de ønskede kolonner. Så hvordan gør vi dette?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) giver os mulighed for at *vælge* (og eventuelt omdøbe) kolonner i en data frame.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Trænings- og testdata

Det er almindelig praksis inden for superviseret læring at *opdele* dataene i to undergrupper; et (typisk større) sæt til at træne modellen med, og et mindre "tilbageholdt" sæt til at evaluere, hvordan modellen klarer sig.

Nu hvor vi har dataene klar, kan vi undersøge, om en maskine kan hjælpe med at finde en logisk opdeling mellem tallene i dette datasæt. Vi kan bruge pakken [rsample](https://tidymodels.github.io/rsample/), som er en del af Tidymodels-rammeværket, til at oprette et objekt, der indeholder information om *hvordan* dataene skal opdeles, og derefter to yderligere rsample-funktioner til at udtrække de oprettede trænings- og testsæt:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Træn en lineær regressionsmodel med Tidymodels

Nu er vi klar til at træne vores model!

I Tidymodels specificerer du modeller ved hjælp af `parsnip()` ved at angive tre begreber:

-   Model **type** adskiller modeller som lineær regression, logistisk regression, beslutningstræ-modeller og så videre.

-   Model **mode** inkluderer almindelige muligheder som regression og klassifikation; nogle modeltyper understøtter begge dele, mens andre kun har én tilstand.

-   Model **engine** er det beregningsværktøj, der vil blive brugt til at tilpasse modellen. Ofte er disse R-pakker, såsom **`"lm"`** eller **`"ranger"`**

Denne modelinformation fanges i en modelspecifikation, så lad os bygge en!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Efter en model er blevet *specificeret*, kan modellen `estimeres` eller `trænes` ved hjælp af [`fit()`](https://parsnip.tidymodels.org/reference/fit.html)-funktionen, typisk ved brug af en formel og nogle data.

`y ~ .` betyder, at vi vil tilpasse `y` som den forudsagte værdi/mål, forklaret af alle prædiktorer/egenskaber, dvs. `.` (i dette tilfælde har vi kun én prædiktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Fra modeloutputtet kan vi se de koefficienter, der blev lært under træningen. De repræsenterer koefficienterne for den bedste tilpasningslinje, som giver os den laveste samlede fejl mellem den faktiske og den forudsagte variabel.
<br>

## 5. Lav forudsigelser på testdatasættet

Nu hvor vi har trænet en model, kan vi bruge den til at forudsige sygdomsprogressionen y for testdatasættet ved hjælp af [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Dette vil blive brugt til at tegne linjen mellem datagrupper.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Vi har lige trænet en model og brugt den til at lave forudsigelser!

Når man laver forudsigelser, er tidymodels-konventionen altid at producere en tibble/data frame med resultater og standardiserede kolonnenavne. Dette gør det nemt at kombinere de originale data og forudsigelserne i et brugbart format til efterfølgende operationer såsom visualisering.

`dplyr::bind_cols()` binder effektivt flere data frames kolonnevis.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Visualisering af modelleringsresultater

Nu er det tid til at se dette visuelt 📈. Vi laver et scatterplot af alle `y`- og `bmi`-værdierne fra testdatasættet og bruger derefter forudsigelserne til at tegne en linje på det mest passende sted mellem modellens datagrupperinger.

R har flere systemer til at lave grafer, men `ggplot2` er en af de mest elegante og alsidige. Det giver dig mulighed for at sammensætte grafer ved **at kombinere uafhængige komponenter**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Tænk lidt over, hvad der sker her. En lige linje løber gennem mange små datapunkter, men hvad gør den egentlig? Kan du se, hvordan du burde kunne bruge denne linje til at forudsige, hvor et nyt, uset datapunkt skulle passe i forhold til plottets y-akse? Prøv at sætte ord på den praktiske anvendelse af denne model.

Tillykke, du har bygget din første lineære regressionsmodel, lavet en forudsigelse med den og vist den i et plot!



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
